이번 Lab을 통해 XOR 문제를 다층 퍼셉트론으로 구현해보겠습니다!

In [1]:
import torch
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

XOR 문제를 풀기 위한 입력과 출력을 정의해줍니다.

In [7]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

입력층, 은닉층 3개, 출력층을 가지는 인공신경망을 구현해보겠습니다.

In [8]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input layer의 크기를 생각해보세요.
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # output layer의 크기를 생각해보세요.
          nn.Sigmoid()
          ).to(device)

In [9]:
"""
# 이렇게 구현해도 됩니다.
# nn layers
linear1 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True) # input layer의 크기를 생각해보세요.
linear2 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True)
linear3 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True)
linear4 = torch.nn.Linear(#이 부분을 채워넣으세요, #이 부분을 채워넣으세요, bias=True) # output layer의 크기를 생각해보세요.
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)
"""

'\n# 이렇게 구현해도 됩니다.\n# nn layers\nlinear1 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True) # input layer의 크기를 생각해보세요.\nlinear2 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True)\nlinear3 = torch.nn.Linear(#이 부분을 채워넣으세요, 10, bias=True)\nlinear4 = torch.nn.Linear(#이 부분을 채워넣으세요, #이 부분을 채워넣으세요, bias=True) # output layer의 크기를 생각해보세요.\nsigmoid = torch.nn.Sigmoid()\n\n# model\nmodel = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)\n'

이제 비용 함수와 옵타마이저를 선언합니다.

In [10]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device) # 이진 분류에서 사용하는 크로스엔트로피 함수(Binary Cross Entropy)를 사용하겠습니다.
optimizer = torch.optim.SGD(model.parameters(), lr=1) # SGD optimizer를 사용하고 learning rate는 1로 하겠습니다.

총 10,001번의 에포크를 수행합니다. 각 에포크마다 역전파가 수행된다고 보면 되겠습니다.

In [11]:
for step in range(10001):
    optimizer.zero_grad()  # gradient를 0으로 초기화
    hypothesis = model(X) # 모델을 생성해서 hypothesis를 구해보세요.

    # cost/loss function
    cost = criterion(hypothesis, Y) # 위에서 정의한 비용함수를 사용하세요.
    cost.backward()  # 비용 함수를 미분하여 gradient 계산
    optimizer.step() # W와 b를 업데이트

    if step % 100 == 0:
        print(step, cost.item())

0 0.7408678531646729
100 0.6931207776069641
200 0.6931231021881104
300 0.6931334137916565
400 0.6931451559066772
500 0.6931242942810059
600 0.69315105676651
700 0.6931241154670715
800 0.6931461691856384
900 0.6931164264678955
1000 0.6931250095367432
1100 0.6931077241897583
1200 0.6931201815605164
1300 0.6931212544441223
1400 0.6931224465370178
1500 0.6931343674659729
1600 0.6931078433990479
1700 0.6931255459785461
1800 0.6930794715881348
1900 0.6930876970291138
2000 0.6930941343307495
2100 0.6930955648422241
2200 0.6930923461914062
2300 0.6930643916130066
2400 0.6930633783340454
2500 0.6930414438247681
2600 0.693047046661377
2700 0.6930262446403503
2800 0.6930091381072998
2900 0.6929957866668701
3000 0.6929850578308105
3100 0.6929338574409485
3200 0.692891001701355
3300 0.6928637027740479
3400 0.6928143501281738
3500 0.6927416324615479
3600 0.6926450729370117
3700 0.6925110816955566
3800 0.6922191381454468
3900 0.6918760538101196
4000 0.6910688877105713
4100 0.689216136932373
4200 0.68

이제 모델이 XOR 문제를 풀 수 있는지 테스트 해봅시다.

In [12]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[6.9727197e-05]
 [9.9988103e-01]
 [9.9989235e-01]
 [1.5759321e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


실제값은 0, 1, 1, 0이며 예측값은 0, 1, 1, 0으로 문제를 해결하는 모습을 보여줍니다.

Q1. Backpropagation이란?

=> 역순으로 진행하면서 오차를 이용해 가중치를 업데이트하는 방법

Q2. Backpropagation은 어떤 방식을 이용해 weight, bias 같은 변수들을 업데이트 할까요?

=> Chain Rule을 이용하여 업데이트한다. 